In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import TextVectorization,LSTM,Dropout,Embedding,Dense,Bidirectional,Input
from keras.layers import CuDNNLSTM

In [ ]:
train_df = pd.read_csv('data/train.csv')
test_x =pd.read_csv('data/test.csv') 
test_y =pd.read_csv('data/test_labels.csv')
test_x.head()

In [ ]:
test_df=pd.merge(test_x,test_y,how='inner',on='id')


In [ ]:
test_df.shape

In [ ]:
test_df =test_df[test_df["toxic"]!=-1].reset_index(drop=True)

test_df.head(5)

In [ ]:
test_df.shape

In [ ]:
train_df =train_df.sample(n=20000, random_state=42)
test_df =test_df.sample(n=4000, random_state=42)


In [ ]:
train_df.info()
train_df.head(5)

In [ ]:
test_df.info()
test_df.head()

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(train_df)
plt.show()


In [ ]:
X_train =train_df['comment_text']
y_train=train_df[train_df.columns[2:]].values

In [ ]:
X_test =test_df['comment_text']
y_test =test_df[test_df.columns[2:]].values

In [ ]:
X_train.head()

In [ ]:
max_length =200000

In [ ]:
vectorizer =TextVectorization(max_tokens =max_length,output_sequence_length=1800,output_mode='int')
vectorizer.adapt(X_train.values)
vectorizer_text =vectorizer(X_train.values)

In [ ]:
vectorizer1 =TextVectorization(max_tokens = max_length,output_sequence_length=1800,output_mode ='int')
vectorizer1.adapt(X_test.values)
vectorizer1_text =vectorizer1(X_test.values)

In [ ]:
data_set = tf.data.Dataset.from_tensor_slices((vectorizer_text, y_train))
data_set = data_set.cache()
data_set = data_set.shuffle(160000)
data_set = data_set.batch(16)
data_set = data_set.prefetch(8)

In [ ]:
test_set =tf.data.Dataset.from_tensor_slices((vectorizer1_text,y_test))
test_set=test_set.cache()
test_set=test_set.shuffle(160000)
test_set=test_set.batch(16)
test_set=test_set.prefetch(8)

In [ ]:
model = Sequential([
    Embedding(max_length+1,32),
    Bidirectional(CuDNNLSTM(32)),
    Dense(128,activation ='relu'),
    Dense(256,activation ='relu'),
    Dense(128,activation ='relu'),
    Dense(6,activation ='sigmoid'),
])
model.compile(loss = 'BinaryCrossentropy',optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(data_set,epochs=1)

In [ ]:
loss,acc =model.evaluate(test_set)

In [ ]:
import numpy as np
text = ["Yo bitch fuck fuck fuck cock you black bitch"]
text_vect = TextVectorization(max_tokens = max_length,output_sequence_length=1800,output_mode ='int')
text_vect.adapt(text)
text_vect_text =text_vect(text)

In [ ]:

predictions = model.predict(text_vect_text) > 0.5
prediction_text = ""
for i, col in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
    prediction_text += f"{col:<{20}}: {predictions[0][i]:<{5}}\n"
print(prediction_text)

In [48]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"  # Example: BERT base uncased
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Example text
text = "This is a sample text for classification."

# Tokenize text
inputs = tokenizer(text, return_tensors="tf", padding=True, truncation=True)

# Perform inference
outputs = model(inputs)
predictions = tf.nn.softmax(outputs.logits, axis=-1)

print("Predictions:", predictions)


c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rvaib\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate dev

Predictions: tf.Tensor([[0.6538972  0.34610283]], shape=(1, 2), dtype=float32)
